In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from datetime import datetime
from sklearn.model_selection import StratifiedKFold
from xgboost import XGBClassifier

In [2]:
def average(df):
    sum = 0
    count = 0
    for x in df['f1']:
        if x!=-1:
            sum+=x
            count+=1
    avg = sum/count
    return avg


In [3]:
#Timer function
def timer(start_time=None):
    if not start_time:
        start_time = datetime.now()
        return start_time
    elif start_time:
        thour, temp_sec = divmod((datetime.now() - start_time).total_seconds(), 3600)
        tmin, tsec = divmod(temp_sec, 60)
        print('\n Time taken: %i hours %i minutes and %s seconds.' % (thour, tmin, round(tsec, 2)))

In [4]:
df = pd.read_csv("train_final.csv")
df.describe()
f1_avg = average(df)
df['f1'].replace(-1,f1_avg, inplace = True)
df.describe()

,Id,Y,f1,f2,f3,f4,f5,f6,f7,f8,...,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24
count,16383.000000,16383.000000,16383.000000,16383.000000,16383.000000,16383.000000,16383.000000,16383.000000,16383.000000,16383.000000,...,16383.000000,16383.000000,16383.000000,16383.000000,16383.000000,16383.000000,16383.000000,16383.000000,1.638300e+04,16383.000000
mean,8192.000000,0.942135,43031.415720,1.044375,11.770938,118323.581456,1.044436,0.050052,117089.674113,169730.178600,...,25894.316914,119045.099005,184622.040835,1.047305,125959.667765,1.044558,1.045718,1.041934,3.271890e+04,1.043948
std,4729.509065,0.233495,33596.053696,0.264806,353.187115,4518.059755,0.265601,0.293892,10261.292970,69396.677853,...,36086.993946,18321.987129,100590.811845,0.306239,31091.344158,0.262576,0.266874,0.246597,3.184929e+06,0.259640
min,1.000000,0.000000,37.000000,1.000000,1.770000,23779.000000,1.000000,0.000000,4292.000000,4673.000000,...,25.000000,4674.000000,3130.000000,1.000000,117879.000000,1.000000,1.000000,1.000000,1.000000e+00,1.000000
25%,4096.500000,1.000000,20331.000000,1.000000,1.770000,118096.000000,1.000000,0.000000,117961.000000,117906.000000,...,4554.000000,118395.000000,118398.000000,1.000000,118274.000000,1.000000,1.000000,1.000000,1.000000e+00,1.000000
50%,8192.000000,1.000000,35530.000000,1.000000,1.770000,118300.000000,1.000000,0.000000,117961.000000,128130.000000,...,13234.000000,118929.000000,119095.000000,1.000000,118568.000000,1.000000,1.000000,1.000000,2.000000e+00,1.000000
75%,12287.500000,1.000000,74240.500000,1.000000,3.540000,118386.000000,1.000000,0.000000,117961.000000,234498.500000,...,38902.000000,120539.000000,290919.000000,1.000000,120006.000000,1.000000,1.000000,1.000000,9.000000e+00,1.000000
max,16383.000000,1.000000,312152.000000,7.000000,43910.160000,286791.000000,9.000000,10.000000,311178.000000,311867.000000,...,311696.000000,286792.000000,308574.000000,18.000000,311867.000000,8.000000,8.000000,7.000000,4.042886e+08,8.000000


In [6]:
y = df.loc[:,'Y']
X = df.loc[:,'f1':'f24']
X_train, X_valid, y_train, y_valid = train_test_split(X,y,train_size = 0.75, test_size = 0.25,random_state = 42, shuffle = True)

In [7]:
model_xgb = XGBClassifier(n_estimators=800, max_depth=4, learning_rate=0.05, n_jobs=-1, random_state=42) 
model_xgb.fit(X_train, y_train)
soft_probs = model_xgb.predict_proba(X_valid)
valid_auc = roc_auc_score(y_valid, soft_probs[:,1])
print(valid_auc)
train_preds = model_xgb.predict(X_train)
acc = accuracy_score(y_train, train_preds)
print(acc)
preds = model_xgb.predict(X_valid)
acc = accuracy_score(y_valid, preds)
print(acc)

0.8648734965318772
0.9721657035891593
0.961181640625


Tuning without feature engineering

In [7]:
params = {
        'min_child_weight': [1, 5, 10],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.8, 0.9, 1.0],
        'max_depth': [4, 5, 6],
        'n_estimators': [800, 1000, 1200]
        }

In [15]:
xgb = XGBClassifier(learning_rate=0.05,
                    silent=True, nthread=1)

In [17]:
# folds = 5
# param_comb = 20 #number of random parameter combos to pick
# skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 42)
# grid_search = GridSearchCV(xgb, param_grid=params,scoring='roc_auc', 
#                             n_jobs=-1, cv=skf.split(X_train,y_train),verbose=3)
# start_time = timer(None) # timing starts from this point for "start_time" variable
# model4 = grid_search.fit(X_train, y_train)
# timer(start_time) # timing ends here for "start_time" variable

Fitting 5 folds for each of 324 candidates, totalling 1620 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:   39.1s
[Parallel(n_jobs=-1)]: Done 112 tasks      | elapsed:  4.2min
[Parallel(n_jobs=-1)]: Done 272 tasks      | elapsed: 11.4min
[Parallel(n_jobs=-1)]: Done 496 tasks      | elapsed: 23.7min
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed: 37.5min
[Parallel(n_jobs=-1)]: Done 1136 tasks      | elapsed: 59.7min
[Parallel(n_jobs=-1)]: Done 1552 tasks      | elapsed: 87.2min
[Parallel(n_jobs=-1)]: Done 1620 out of 1620 | elapsed: 92.8min finished


[19:31:38] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.



 Time taken: 1 hours 33 minutes and 3.72 seconds.


In [18]:
soft_probs = model4.predict_proba(X_valid)
valid_auc = roc_auc_score(y_valid, soft_probs[:,1])
print(valid_auc)
train_preds = model4.predict(X_train)
acc = accuracy_score(y_train, train_preds)
print(acc)
preds = model4.predict(X_valid)
acc = accuracy_score(y_valid, preds)
print(acc)

0.875433515348288
0.9798160657605599
0.9619140625


In [19]:
print(model4.best_params_)

{'colsample_bytree': 0.6, 'max_depth': 4, 'min_child_weight': 1, 'n_estimators': 1000, 'subsample': 0.8}


{'colsample_bytree': 0.6, 'max_depth': 4, 'min_child_weight': 1, 'n_estimators': 1000, 'subsample': 0.8} These are the results from gridsearch

Slight improvement but the grid search was not worth it. Lets just submit this for the sake of a submission. 

In [22]:
testdf = pd.read_csv("test_final.csv")
testdf['f1'].replace(-1,f1_avg, inplace = True)
X_test = testdf.loc[:,'f1':'f24']
#X_test = scaler.transform(X_test)
test_preds = model_grid.predict_proba(X_test)
sub3 = {"Id":testdf['Id'],"Y":test_preds[:,1]}
sub3 = pd.DataFrame(data=sub3)
sub3.to_csv("submissions/sub3.csv", index=False)

Submission improved score from previous XGBoost. from 0.87347 to 0.88364

In [8]:
model_grid = XGBClassifier(n_estimators = 1000,colsample_bytree=0.6,learning_rate=0.05, njobs=-1,
                    max_depth = 4, random_state=42)
model_grid.fit(X_train, y_train)

[20:44:08] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { njobs } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.6, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.05, max_delta_step=0, max_depth=4,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=1000, n_jobs=0, njobs=-1, num_parallel_tree=1,
              random_state=42, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
              subsample=1, tree_method='exact', validate_parameters=1,
              verbosity=None)

In [9]:
soft_probs = model_grid.predict_proba(X_valid)
valid_auc = roc_auc_score(y_valid, soft_probs[:,1])
print(valid_auc)
train_preds = model_grid.predict(X_train)
acc = accuracy_score(y_train, train_preds)
print(acc)
preds = model_grid.predict(X_valid)
acc = accuracy_score(y_valid, preds)
print(acc)

0.8714903704250295
0.9750142426955318
0.961669921875


In [10]:
#What if I normalize all data and use it
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
model_grid.fit(X_train, y_train)

[20:44:24] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { njobs } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.6, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.05, max_delta_step=0, max_depth=4,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=1000, n_jobs=0, njobs=-1, num_parallel_tree=1,
              random_state=42, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
              subsample=1, tree_method='exact', validate_parameters=1,
              verbosity=None)

In [11]:
soft_probs = model_grid.predict_proba(X_valid)
valid_auc = roc_auc_score(y_valid, soft_probs[:,1])
print(valid_auc)
train_preds = model_grid.predict(X_train)
acc = accuracy_score(y_train, train_preds)
print(acc)
preds = model_grid.predict(X_valid)
acc = accuracy_score(y_valid, preds)
print(acc)

0.8714938293240849
0.9750142426955318
0.961669921875


Normalizing did not make ANY significant changes to the scores. 